This notebook runs one game of connectx, using a Monte Carlo tree search agent for one player (blue) and a 3 turn lookahead agent for the other (grey).

In [ ]:
from kaggle_environments import make, evaluate, utils
import matplotlib.pyplot as plt

import lookahead_agent
import mcts_agent

## Basic Usage
To use the agent, call mcts_agent.mcts_agent and pass an observation and configuration, matching the format defined in the Kaggle competiton. By default, the agent will use multiprocessing.

Single process and multi process agents can be called directly as mcts_agent.sp_mcts_agent and mcts_agent.mp_mcts_agent.

The following cell shows how to use the agent, run one game of Connect X, and render a replay of the game.

In [ ]:
# run one game, can take a few minutes
env = make("connectx")
res = env.run([
    mcts_agent.mcts_agent,
    lookahead_agent.three_lookahead_agent
])
env.render(mode="ipython")

## Additional visualization
It may be interesting to see how the MCTS agent thinks the game will end. We measure this using the results of the playouts during the agent's turn. Because the agent stores limited information from previous turns, we need to write a wrapper to capture this information each turn.

In [ ]:
outcomes = {}
def outcome_wrapper(obs, cfg):
    action = mcts_agent.mcts_agent(obs, cfg)
    if mcts_agent.mp_or_sp == "sp":
        agent = mcts_agent.my_mcts
    else:
        agent = mcts_agent.my_mctss[0]
    visits = agent.root.visits
    turn_outcomes = agent.root.wins
    wins = turn_outcomes[obs["mark"]] / visits
    losses = turn_outcomes[obs["mark"]%2+1] / visits
    ties = (visits - (turn_outcomes[obs["mark"]] + turn_outcomes[obs["mark"]%2+1])) / visits
    outcomes[obs["step"]] = [wins, losses, ties]
    return action

env = make("connectx", debug=True)
res = env.run([
    outcome_wrapper,
    lookahead_agent.three_lookahead_agent
])
env.render(mode="ipython")

In [ ]:
turns = list(outcomes.keys())
wins, losses, ties = zip(*outcomes.values())
plt.plot(turns, wins, c="tab:green", label="Agent")
plt.plot(turns, losses, c="tab:red", label="Opponent")
plt.plot(turns, ties, c="tab:grey", label="Tie")
plt.legend()
plt.title("Expected result")
plt.xlabel("Turn")
plt.ylabel("Probability")
plt.grid()